In [40]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import t,norm,sem,ttest_ind 
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf #same functionality as R's lm
import statsmodels.api as sm
import seaborn; seaborn.set(style='whitegrid');
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from fancyimpute import IterativeImputer

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [41]:
data = pd.read_csv("data_cleaned.csv", index_col = 0)
columns = data.columns.tolist()
data = data[data['soc'].notna()]
size = data.shape[0]
print(size)

11982


/usr/local/Cellar/jupyterlab/2.1.4/libexec/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [42]:
dataw = pd.read_csv("data_cleaned_with_weather.csv", index_col = 0)

In [43]:
weather = ['TS',
 'T2M_RANGE',
 'T2MDEW',
 'T2MWET',
 'T2M_MAX',
 'T2M_MIN',
 'RH2M',
 'PS',
 'PRECTOT',
 'QV2M',
 'T2M']

In [44]:
weather

['TS',
 'T2M_RANGE',
 'T2MDEW',
 'T2MWET',
 'T2M_MAX',
 'T2M_MIN',
 'RH2M',
 'PS',
 'PRECTOT',
 'QV2M',
 'T2M']

In [45]:
data['soc']

1             -
2        134.17
3        112.92
4        107.43
5         97.92
          ...  
11978      0.01
11979      0.01
11980      0.01
11981      0.01
11982      0.01
Name: soc, Length: 11982, dtype: object

In [54]:
num_variables = ['layer_top','layer_bot','total_lcount','soc_depth','elevation',
                'slope','B1','B2','B3','B4','B5','B6','B7','precip']
cat_variables = columns[52:-15]
all_variables = num_variables + cat_variables

In [47]:
data_x = data[all_variables]
data_y = data['soc']

In [48]:
def count_nan(column, df):
    na_count = df[column].isna().sum()
    return na_count

In [49]:
#print number of na's for each variable
#for name in all_variables:
#    print(name)
#    print(count_nan(name, data_x))

In [90]:
mice_imputer = IterativeImputer() 
data_x = mice_imputer.fit_transform(data_x) 
data_x = pd.DataFrame(data_x)
data_x.columns = all_variables
data_x

/usr/local/Cellar/jupyterlab/2.1.4/libexec/lib/python3.8/site-packages/sklearn/impute/_iterative.py:669: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


,layer_top,layer_bot,total_lcount,carbon_lcount,soc_lcount,soc_depth,elevation,slope,B1,B2,...,soc_lcount_6,soc_lcount_7,soc_lcount_8,soc_lcount_9,soc_lcount_10,soc_lcount_11,soc_lcount_12,soc_lcount_13,soc_lcount_14,soc_lcount_NA
0,0.0,1.0,1.0,1.0,1.0,1.0,1591.000000,2.000000,15165.000000,26255.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,165.0,3.0,3.0,3.0,165.0,385.000000,1.000000,814.241071,1572.812500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,156.0,4.0,4.0,4.0,156.0,466.000000,0.000000,1581.904762,3530.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,200.0,8.0,8.0,8.0,200.0,365.771683,2.000000,4633.528037,5882.733645,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,203.0,5.0,5.0,5.0,203.0,270.000000,1.000000,2955.103093,3600.670103,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11977,0.0,50.0,2.0,2.0,2.0,50.0,497.639709,11.720993,69226.254300,51550.137460,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11978,0.0,10.0,4.0,4.0,4.0,10.0,1338.000000,3.000000,21420.000000,34830.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11979,0.0,11.0,2.0,2.0,2.0,11.0,1643.000000,17.000000,23100.648650,38889.135140,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11980,0.0,3.0,1.0,1.0,1.0,3.0,1338.000000,4.000000,25950.000000,42877.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
data_x.to_csv('data_x_mice.csv')

In [79]:
data_x = pd.read_csv('data_x_mice.csv')

In [105]:
#drop the first entry where the soc is "-"
data_x = data_x.drop([1])


KeyError: '[1] not found in axis'

In [118]:
data_y = data['soc']
data_y = data_y.drop([1])

In [119]:
data_x = data_x[all_variables] #to remove the soc lcounts
data_x.index = data.index.tolist()[1:]

In [120]:
data_x.to_csv('data_x_clean_noweather.csv')
data_y.to_csv('data_y_clean.csv')

In [121]:
data_x = pd.read_csv('data_x_clean_noweather.csv',index_col = 0)
data_y = pd.read_csv('data_y_clean.csv',index_col = 0)
data_y

,soc
2,134.17
3,112.92
4,107.43
5,97.92
6,96.28
...,...
11978,0.01
11979,0.01
11980,0.01
11981,0.01


In [122]:
#impute weather
w_columns = dataw[weather].columns.tolist()
mice_imputer = IterativeImputer()
dataw[weather] = mice_imputer.fit_transform(dataw[weather]) 
dataw[weather] = pd.DataFrame(dataw[weather])
dataw[weather].columns = w_columns

In [123]:
dataw[weather]

,TS,T2M_RANGE,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,RH2M,PS,PRECTOT,QV2M,T2M
1,28.30,13.04,10.76,10.76,34.05,21.01,36.15,90.53,0.07,9.03,27.24
2,24.19,11.58,16.10,16.10,29.49,17.91,64.67,101.22,0.03,11.40,23.23
3,25.05,7.27,17.02,17.02,27.70,20.44,66.21,101.45,0.15,12.06,23.80
4,10.54,11.13,5.16,5.17,16.76,5.63,69.66,98.01,0.61,5.64,10.50
5,11.94,8.41,6.64,6.64,16.29,7.88,70.83,97.56,1.77,6.27,11.77
...,...,...,...,...,...,...,...,...,...,...,...
13050,6.71,11.77,0.91,0.95,15.36,3.59,65.42,99.89,0.01,4.13,7.12
13051,6.71,11.77,0.91,0.95,15.36,3.59,65.42,99.89,0.01,4.13,7.12
13052,6.71,11.77,0.91,0.95,15.36,3.59,65.42,99.89,0.01,4.13,7.12
13053,6.71,11.77,0.91,0.95,15.36,3.59,65.42,99.89,0.01,4.13,7.12


In [124]:
#add weather data to x data
for w in weather:
    data_x[w] = None 
for i in data_x.index.tolist():
    for w in weather:
        data_x[w][i] = dataw[w][i]

<ipython-input-124-ed4fa9ab8c36>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_x[w][i] = dataw[w][i]


In [126]:
data_x.to_csv('data_x_clean_weather.csv')

In [39]:
data_x.columns

Index(['layer_top', 'layer_bot', 'total_lcount', 'carbon_lcount', 'soc_depth',
       'elevation', 'slope', 'B1', 'B2', 'B3',
       ...
       'drainagecl_very.poorly', 'drainagecl_well', 'drainagecl_NA',
       'runoff_high', 'runoff_low', 'runoff_medium', 'runoff_negligible',
       'runoff_very.high', 'runoff_very.low', 'runoff_NA'],
      dtype='object', length=205)

In [71]:
weather

['TS',
 'T2M_RANGE',
 'T2MDEW',
 'T2MWET',
 'T2M_MAX',
 'T2M_MIN',
 'RH2M',
 'PS',
 'PRECTOT',
 'QV2M',
 'T2M']

In [127]:
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.1, random_state=42)

In [128]:
X_train[num_variables] = X_train[num_variables].astype(float)

/usr/local/Cellar/jupyterlab/2.1.4/libexec/lib/python3.8/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [129]:
y_train = y_train.astype(float)

In [130]:
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_train, y_train)

0.20562749933517066

In [131]:
reg.score(X_test,y_test)

0.19379399901760463

In [64]:
reg.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}

In [65]:
reg.coef_

array([-9.33227154e-02, -2.67906340e-04, -1.32796012e-01,  1.48580707e-02,
        5.99815283e-04, -1.95626698e-03, -9.79863604e-06, -5.73753259e-05,
        4.49043029e-05,  6.86469650e-06, -1.93664357e-05, -1.41282791e-06,
        1.15596989e-05,  3.77666460e-03, -2.31696592e+00, -2.40639615e+00,
        2.71977923e+02, -1.16890162e+00, -1.68038398e+00, -2.23320522e-01,
       -2.52118348e+00, -1.84396254e+00, -3.82976537e+02, -2.00395591e+00,
       -4.96999512e-01, -2.48693617e+00, -4.51496055e+00, -2.84399377e+00,
       -1.01995398e+00, -2.97793366e+00, -1.81494596e+00, -8.79136968e-01,
       -1.80273164e+00, -5.54688534e+00, -1.37924095e+00, -1.60589733e+00,
       -1.26755587e+00, -2.08926143e+00, -2.05506348e-01, -1.54758425e+00,
        3.32714847e-01,  2.04877750e+02,  5.19425566e-01, -1.09831093e+00,
       -2.54797128e+00, -3.78981980e+00,  4.58196373e-01, -4.40432678e+00,
       -1.65501710e+00, -7.35376427e-01, -3.46579225e+00, -1.11605855e+01,
       -4.33882334e+00,  

In [132]:
#X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
regr = RandomForestRegressor(max_depth=5, random_state=0)
regr.fit(X_train, y_train)
#print(regr.predict([[0, 0, 0, 0]]))

<ipython-input-132-3786324d614c>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(X_train, y_train)


RandomForestRegressor(max_depth=5, random_state=0)

In [133]:
regr.predict(X_test)

array([3.2181772 , 3.84405214, 2.09463962, ..., 3.78184371, 2.96067684,
       4.00958171])

In [135]:
regr.score(X_train,y_train)

0.49084729109390823

In [134]:
regr.score(X_test,y_test)

0.18793897835423512

In [51]:
regr.get_params(deep=True)

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 5,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

In [52]:
regr.apply(X_train)

array([[53, 53, 53, ..., 54, 47, 53],
       [59, 59, 59, ..., 61, 53, 59],
       [50, 50, 50, ..., 47, 44, 50],
       ...,
       [57, 57, 56, ..., 55, 51, 57],
       [44, 44, 44, ..., 39, 38, 44],
       [59, 59, 57, ..., 59, 53, 59]])

In [69]:
#formula_all = 'soc ~ ' + ' + '.join(all_variables)
#print(formula_all)

In [70]:
#reg_setup_1 =smf.ols(formula=formula_all, data=train)
#model_1=reg_setup_1.fit()
#print(model_1.summary())

['state_Alabama',
 'state_Arizona',
 'state_Arkansas',
 'state_California',
 'state_Colorado',
 'state_Connecticut',
 'state_Delaware',
 'state_District.of.Columbia',
 'state_Florida',
 'state_Georgia',
 'state_Idaho',
 'state_Illinois',
 'state_Indiana',
 'state_Iowa',
 'state_Kansas',
 'state_Kentucky',
 'state_Louisiana',
 'state_Maine',
 'state_Maryland',
 'state_Massachusetts',
 'state_Michigan',
 'state_Minnesota',
 'state_Mississippi',
 'state_Missouri',
 'state_Montana',
 'state_Nebraska',
 'state_Nevada',
 'state_New.Hampshire',
 'state_New.Jersey',
 'state_New.Mexico',
 'state_New.York',
 'state_North.Carolina',
 'state_North.Dakota',
 'state_Ohio',
 'state_Oklahoma',
 'state_Oregon',
 'state_Pennsylvania',
 'state_Rhode.Island',
 'state_South.Carolina',
 'state_South.Dakota',
 'state_Tennessee',
 'state_Texas',
 'state_Utah',
 'state_Vermont',
 'state_Virginia',
 'state_Washington',
 'state_West.Virginia',
 'state_Wisconsin',
 'state_Wyoming',
 'soc_carbon_flag_Complete',
 '

In [33]:
X_train['vegclass_local']

10193    NaN
4036     NaN
9881     NaN
11188    NaN
3976     NaN
        ... 
11965    NaN
5192     NaN
5391     NaN
861      NaN
7271     NaN
Name: vegclass_local, Length: 10783, dtype: object

In [28]:
X_train.head()

,dataset_name_sub,dataset_name_soc,datum,state,country,observation_date,site_name,profile_name,profile_zero_ref,layer_top,...,soc_lcount_6,soc_lcount_7,soc_lcount_8,soc_lcount_9,soc_lcount_10,soc_lcount_11,soc_lcount_12,soc_lcount_13,soc_lcount_14,soc_lcount_NA
10193,NRCS Sept/2014,ISCN SOC stock to 1m computation,WGS84,Tennessee,United States,11/7/2006,S2006TN061042,07N0181,topO,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4036,NRCS Sept/2014,ISCN SOC stock computation,WGS84,Washington,United States,8/19/1991,S1991WA047007,92P0399,topO,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9881,NRCS Sept/2014,ISCN SOC stock computation,WGS84,Tennessee,United States,1/2/1999,S1999TN085002,00P0250,topO,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
11188,NRCS Sept/2014,ISCN SOC stock computation,WGS84,South Carolina,United States,9/20/2006,S2006SC021006,07N0199,topO,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3976,NRCS Sept/2014,ISCN SOC stock computation,WGS84,California,United States,8/2/1999,99CA693-032,00P0350,topO,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
